This script using skggm to compute partial correlation of EEG amplitude

In [1]:
# load modules
from os import listdir

from hdf5storage import loadmat, savemat 
import numpy as np 
from matplotlib import pyplot as plt

from scipy import signal 
from scipy.fftpack import fft
from scipy import stats # for zscoring

from sklearn.preprocessing import StandardScaler
from sklearn.covariance import GraphicalLassoCV

/home/zhibinz2/anaconda3/envs/eeg/lib/python3.9/site-packages/sklearn/utils/multiclass.py:13: DeprecationWarning: Please use `spmatrix` from the `scipy.sparse` namespace, the `scipy.sparse.base` namespace is deprecated.
  from scipy.sparse.base import spmatrix
/home/zhibinz2/anaconda3/envs/eeg/lib/python3.9/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/home/zhibinz2/anaconda3/envs/eeg/lib/python3.9/site-packages/sklearn/linear_model/least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, 

In [2]:
# sklearn version
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))
# confirm that it is version 0.21.3 (The version that works)


The scikit-learn version is 0.21.3.


In [4]:
# add this repo to your path (https://github.com/skggm/skggm) （ I'm adding my fork https://github.com/zhibinz2/skggm in the following)
import sys
sys.path.append("/home/zhibinz2/Documents/GitHub/skggm/inverse_covariance/")
from inverse_covariance import (
    QuicGraphicalLasso,
    QuicGraphicalLassoCV,
    QuicGraphicalLassoEBIC,
    AdaptiveGraphicalLasso,
    ModelAverage,
)


/home/zhibinz2/anaconda3/envs/eeg/lib/python3.9/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [5]:
# skggm (function for computing partial correlation of amplitude)
def partialcorrelation(ampdata,ncv=2):
    # pc_lasso = GraphicalLassoCV(cv=ncv)
    modelquic = QuicGraphicalLassoCV(
        cv=2,  # cant deal w more folds at small size
        n_refinements=6,
        n_jobs=6,
        init_method="cov"
    )
    

    nbin = np.shape(ampdata)[1]
    nchan = np.shape(ampdata)[2]
    pcorr = np.zeros((nbin,nchan,nchan))
    # alphas = np.zeros(nbin)
    covmat_corr = np.zeros((nbin,nchan,nchan))
    for f in range(1,nbin): # one frequency at a time
        xx= np.abs(np.squeeze(ampdata[:,f-1,:]))
        modelquic.fit(xx)

        # covariance = pc_lasso.covariance_
        covariance = modelquic.covariance_
        
        # precision = pc_lasso.precision_
        precision = modelquic.precision_
        
        # alphas[f] = pc_lasso.alpha_
        # alphas[f] = modelquic.alpha_

        parcor = np.zeros((32,32))
        y = np.diag(precision)
        yy = np.outer(y,y)
        yy = np.sqrt(yy)
        parcor = precision/yy
        pcorr[f,:,:] = parcor

        y = np.diag(covariance)
        yy = np.outer(y,y)
        yy = np.sqrt(yy)
        parcor_cov = covariance/yy        
        covmat_corr[f,:,:] = parcor_cov 
    return pcorr,covmat_corr #,alphas  

In [6]:
# load an example EEG  data
outdict=loadmat('trialdata.mat')
trialdata=outdict['trialdata'] # trialdata is 10 epochs of 2000 samples (1 second) by 32 channel EEG data


In [ ]:
# fft
epoch = 1 #units: seconds
maxf = 25 #units: Hz
maxbin = maxf*epoch 
sr=2000 # sampling rate
dataf = fft(trialdata,axis = 1) 
fcoef = dataf[:,1:maxbin+1,:]/(epoch*sr)/epoch
ampdata=np.abs(fcoef)  # amplitude data (below 25 Hz)

# Run graphical lasso for each frequency
pcorr,covmat_corr = partialcorrelation(ampdata,ncv=2)
# pcorr is partial correlation (precision matrix)
# covmat_corr is covariance matrix
# This will take 8 minutes
